In [ ]:
import torch
from hydra import initialize, compose
from hydra.utils import instantiate
from bliss.surveys.dc2 import DC2DataModule
import tqdm

In [ ]:
with initialize(config_path=".", version_base=None):
    notebook_cfg = compose("notebook_config")
notebook_cfg.surveys.dc2.tile_slen = 8
notebook_cfg.surveys.dc2.max_sources_per_tile = 6
notebook_cfg.surveys.dc2.cached_data_path = "./tmp_output/dc2_cached_data"

In [ ]:
dc2: DC2DataModule = instantiate(notebook_cfg.surveys.dc2)
dc2.prepare_data()
dc2.setup(stage="fit")

dc2_train_dataloader = dc2.train_dataloader()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
n_sources_count = torch.zeros(10, device=device)
for batch in tqdm.tqdm(dc2_train_dataloader):
    n_sources_count += batch["tile_catalog"]["n_sources"].to(device=device).flatten().bincount(minlength=10)

In [ ]:
print(n_sources_count)

In [ ]:
print(n_sources_count / n_sources_count.sum())

In [ ]:
print(n_sources_count[1:] / n_sources_count[1:].sum())